In [1]:
from typing import Tuple
import pandas

from prophet import Prophet
from prophet.serialize import model_to_json, model_from_json

from sklearn.metrics import mean_squared_error

/home/ankitapurv/.conda/envs/env-hackathon23_somnolence/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
df = pandas.read_excel(io = '../../data/processed/france.xlsx')

In [3]:
def make_dataset(df_processed: pandas.DataFrame, df_covid_measures: pandas.DataFrame = pandas.DataFrame()) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_mrd = df_processed[['Time', 'Unemployment_Rate_TOT']].rename(
        columns = {'Time': 'ds', 'Unemployment_Rate_TOT': 'y'}
    )
    df_mrd = df_mrd.drop(index = df_mrd[pandas.isnull(df_mrd['y'])].index, inplace = False)
    df_covid_holidays = pandas.DataFrame()
    return df_mrd, df_covid_holidays

In [4]:
df_mrd, _ = make_dataset(df)

In [5]:
def train_test_split(df_mrd: pandas.DataFrame, test_size: int = 12) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_test = df_mrd.tail(test_size)
    df_train = df_mrd.drop(index = df_mrd.tail(test_size).index, inplace = False)
    return df_train, df_test

In [6]:
df_train, df_test = train_test_split(df_mrd, 12)

In [7]:
model = Prophet().fit(df_train)

00:46:15 - cmdstanpy - INFO - Chain [1] start processing
00:46:16 - cmdstanpy - INFO - Chain [1] done processing


In [8]:
def test_model(df_test: pandas.DataFrame, model):
    "return predicted values and rmse"
    df_predicted = model.predict(df_test)
    rmse = mean_squared_error(y_true = df_test['y'], y_pred = df_predicted['yhat'], squared = False)
    return df_predicted, rmse

In [9]:
df_predicted, rmse = test_model(df_test, model)

In [10]:
rmse

0.2588436816437825

In [11]:
with open('france_prophet_benchmark_model.json', 'w') as f:
    f.write(model_to_json(model))

In [12]:
df_predicted

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2022-03-01,7.634662,7.050359,8.225391,7.634662,7.634662,0.021856,0.021856,0.021856,0.021856,0.021856,0.021856,0.0,0.0,0.0,7.656519
1,2022-04-01,7.605190,7.010391,8.247054,7.605190,7.605190,0.008025,0.008025,0.008025,0.008025,0.008025,0.008025,0.0,0.0,0.0,7.613215
2,2022-05-01,7.576667,6.937931,8.171244,7.576667,7.576667,0.001612,0.001612,0.001612,0.001612,0.001612,0.001612,0.0,0.0,0.0,7.578279
3,2022-06-01,7.547194,6.952433,8.205301,7.547194,7.547194,0.008744,0.008744,0.008744,0.008744,0.008744,0.008744,0.0,0.0,0.0,7.555938
4,2022-07-01,7.518672,6.964592,8.134581,7.515973,7.519290,0.039145,0.039145,0.039145,0.039145,0.039145,0.039145,0.0,0.0,0.0,7.557818
5,2022-08-01,7.489199,6.843635,8.155392,7.480382,7.494159,0.036454,0.036454,0.036454,0.036454,0.036454,0.036454,0.0,0.0,0.0,7.525653
6,2022-09-01,7.459726,6.864838,8.133120,7.443417,7.469884,0.052469,0.052469,0.052469,0.052469,0.052469,0.052469,0.0,0.0,0.0,7.512195
7,2022-10-01,7.431204,6.887009,8.069225,7.407520,7.447933,0.054914,0.054914,0.054914,0.054914,0.054914,0.054914,0.0,0.0,0.0,7.486118
8,2022-11-01,7.401731,6.909725,8.103528,7.370183,7.424431,0.076243,0.076243,0.076243,0.076243,0.076243,0.076243,0.0,0.0,0.0,7.477974
9,2022-12-01,7.373209,6.870710,8.047642,7.333758,7.402559,0.072225,0.072225,0.072225,0.072225,0.072225,0.072225,0.0,0.0,0.0,7.445434


In [13]:
df_future = pandas.DataFrame(data = {'ds': ['2023-03-01', '2023-04-01']})

In [14]:
df_future_prediction = model.predict(df_future)

In [15]:
df_future_prediction

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2023-03-01,7.287642,6.691361,8.011997,7.287642,7.287642,0.051844,0.051844,0.051844,0.051844,0.051844,0.051844,0.0,0.0,0.0,7.339486
1,2023-04-01,7.258169,6.662598,7.919193,7.258169,7.258169,0.009988,0.009988,0.009988,0.009988,0.009988,0.009988,0.0,0.0,0.0,7.268157
